<a href="https://colab.research.google.com/github/cahcharm/Neuro-and-Complexity-Science/blob/neuro-morphology/Coding/LTP/correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Structural plasticity controlled by calcium based correlation detection**

## Top tips
- Hebbian rule shows the importance of correlation between pre- and post-synapse.
- The correlation is usually represented by spiking-time but structurally effected by three process: synapse formation, potential/depress, synapse pruning.
- the calcium concentration in spines was experimentally shown to be a correlation sensitive signal. NMDA receptors are a candidate mediator for the correlation dependent calcium signal
- Here they establish a quantitative model of correlation detection in synapses based on the calcium influx through NMDA receptors under realistic conditions of irregular pre- and postsynaptic spiking activity with pairwise correlation.
- find a simple threshold mechanism sensitive to correlation, whch mechanism controls synapse formation and elimination. They show that the connectivity structure is shaped by the correlations between neighboring inputs.
Structural plasticity controlled by calcium based correlation detection

## Basic biochemical process

- Newly formed synapses are created in an intermediate, silent state. These frequently encountered silent synapses lack AMPA receptors but have NMDA receptors. The most probable mechanism is the translocation of AMPA receptors into the postsynaptic density (PSD).
- LTP and LTD are in a bistate dependent on correlation made by spiking. The underlying mechanisms may be similar.
- presynapse Ca2+ release, NMDA activate mechanism, Calcium activate CaMKII, CaMKII phosphorylate AMPA receptor to increase AMPA conductivity and insertion.
- CaMKII forms holoenzymes with 6 subunits triggered by different calcium signal level. Short, weak level activate one subunit and deactivate after the level drops; higher level an active subunit can phosphorylate the neiboring subunit and remain active. At resting calcium level, PP1 can dephosphorylate an active subunit but a neighboring active site can immediately rephosphorylate it again. 
- Calcium also activate PP1 which dephosphorylate CaMKII-- long time scale, bistable of LTP and LTD

## Discussion
- some parameters are not based on experiments, but on analytic convenience
- current model can not reproduce the part of the rule expressing LTD

In [1]:
# if you want to use or haven't installed brainpy 
# pip install -U brain-py

In [12]:
import numpy as np
import brainpy as bp
import matplotlib.pyplot as plt
from scipy.special import gamma
from scipy import integrate

In [2]:
bp.__version__

'1.0.3'

In [3]:
dt=0.5
bp.backend.set(dt=dt)

In [ ]:
# LIF model with delta-shaped postsynaptic currents

## Spike Time Dependence of Postsynaptic Calcium Concentration

- calcium peak amplitude in a spine in good approximation depends exponentially on the temporal difference of the presynaptic and the postsynaptic spiking.
- the calcium influx is largest, if the presynaptic cell fires shortly before the postsynaptic cell. This makes the calcium signal an appropriate
candidate carrier of information on causal correlation

$$
\rho_{glu}, t_{glu}, \tau_{rise,nmda}\\
\Delta t_{syn}=t_{BP} - t_{glu}\\
\Delta t_{syn} = t_{BP} - t_{glu}=t_{post}+d_{BP}-(t_{pre}+d_{glu})=\Delta t + d_{BP}-d_{glu}\\
n(t) = n_0H(t-t_{glu}-\tau_{rise,nmda})e^{-\frac{t-t_{glu}-t_{rise,nmda}}{\tau_{nmda}}},\ 
i_{Ca}=q_0\delta(t-t_{BP})\\
q = \int i_{Ca}(t-t_{BP})n(t) dt = \int q_0\delta(t-t_{BP})n_0H(t-t_{glu}-t_{rise,nmda})e^{-\frac{t-t_{glu}-t_{rise,nmda}}{\tau_{nmda}}} dt\\
= q_0n_0H(\Delta t_{syn}-\tau_{rise,nmda})e^{-\frac{\Delta t_{syn}-t_{rise,nmda}}{\tau_{nmda}}}\\
=q_0n_0H(\Delta t - (\tau_{rise,nmda}+d_{glu}-d_{BP}))e^{-\frac{\Delta t - (\tau_{rise,nmda}+d_{glu}-d_{BP})}{\tau_{nmda}}}\\
=q_0n_0H(\Delta t - \tau_{rise})e^{\frac{\Delta t - \tau_{rise}}{\tau_{nmda}}}\\
q(t_{BP}) = \int_{-\infty}^{\infty} i_{Ca}(t-t_{BP})n(t-t_{glu})dt=\int_0^{\infty} i_{Ca}(t)n(t-t_{glu}+t_{BP})dt\\
=\int i_{Ca}(t)n(t+\Delta t_{syn})dt=\int i_{Ca}(t)e^{-\frac{t}{\tau_{nmda}}} dt e^{-\frac{\Delta t_{syn}-\tau_{rise,nmda}}{\tau_{nmda}}}=q'_0e^{-\frac{\Delta t_{syn}-\tau_{rise,nmda}}{\tau_{nmda}}}(if \Delta t_{syn}\geq \tau_{rise,nmda})
$$

In [9]:
# paraeters setting
rho_glu = 0 # glutamate concentration
t_glu = 0 # time of presynaptic spiking to increase the glutamate concentration
nt = 0 # fraction of the NMDA receptors bind to glutamte
tau_rise_nmda = 5-10 # a rise time n(t) reaches maxium n_0 [ms] 
# we choose this maxium n_0 to be th unit of n, after rho_glu(t) decay back to its resting value, the receptors unbind and n(t) to 0
# we neglect the continuous binding process and instead assume that the number n of glutamate bound NMDA receptors jumps to a positive value after the rise time
n_0 = 1
V_spine = 0 # arriving of a postsynaptic back-propagating action potential(bpAP) depolarizes the spine(V_spine), removes the block and calium ion flow into the spine
t_BP = 5 # arrival time of a postsynaptic bpAP at the spine [ms]
delta_t_syn = t_BP - t_glu # thus receptor mediated conductivity depends on the relative timing delta_t_syn
q = 0 # the total amout of Ca2+ influx, which is proportional to n(t_BP), therefore depends on the relative timing delta_t_syn
t_post = 10 # spiking time of the postsynapse [ms]
t_pre = 0  # spiking time of the presynapse [ms]
delta_t = t_post - t_pre # t_pre spiking time in the presynaptic neuron's soma, t_post spiking time in the postsynaptic neuron
tau_nmda = 32 # time constant of exponential decay [ms]
d_BP = 1 # the delay for bpAP
d_glu =1 # delay between presynaptic spike and glutamte release
q = 0 # total Ca2+ influx
q_0 = 1
tau_rise = 0 # tau_risr_nmda+d_glu-d_BP [ms]


## Ca2+ Transients Caused by Correlated Irregular Spiking

- In this section we investigate how much information about the correlation between presynaptic and postsynaptic events is contained in the Ca2+ signal if the spiking activity is irregular. 
- we simulate the presynaptic and the postsynaptic spiking as Poisson processes with rates $ν_i$ and $ν_o$, respectively. Both processes share a fraction of spikes that appear in the spike trains with a fi xed temporal distance $\Delta t$.
- he information about correlation between the
presynaptic and the postsynaptic events enters the probability
distribution ρ(a) and is therefore exhibited in the amplitude dis-
tribution of Ca2+ transients in the postsynaptic spine

$$
n(t)=n_0\sum e^{-\frac{t-t_{pre}-\tau_{rise}}{\tau_{nmda}}}\\
q_A(t_{post})=\sum e^{-\frac{t_{post}-t_{pre}-\tau_{rise}}{\tau_{nmda}}}\\
\rho_0(q)=Cq^{r-1}(0<q\leq 1), =q^{r-1}(C-r\int_1^{q}\rho_0(x-1)x^{-r}dx)(q>1)\\
r=v_i\tau_{nmda},\ C =\frac{e^{-r\gamma}}{\Gamma(r)},\ \gamma=0.577215665\\
\rho(a)=\varepsilon\rho_{0}(a-\Delta a)+(1-\varepsilon)\rho_{0}(a),\ \Delta a = e^{-(\Delta t-\tau_{rise})/\tau_{nmda}} 
$$

In [10]:
# parameters setting
v_i = 5 # rate of Possion process for presynaptic spiking 
v_o = 5 # rate of Possion process for postsynaptic spiking
epsilon = 0 # correlation coeffient for the strength of the temporal correlation. which is the conditional probability of a pair event
gamma_0 = 0.577215665
N = 80 # average number of CAMKII molecules in a PSD
 

In [ ]:
def probability_distribution(q, v_i):
    '''
    input:
    the Ca influx: q
    Possion efficient of input spike: v_i
    output:
    probability at q: rho_0
    '''
    r = v_i*tau_nmda
    C = np.exp(-r*gamma_0)/gamma(gamma_0)
    if q <= 1:
        rho_0 = C*q**(r-1)
    elif q > 1:
        rho_0 = q**(r-1)*(C-r)
    return rho_0


## A Counter for Correlated Events

- Appendix: Probability Distribution for the Number of Active CaMKII Molecules
- Mean First Passage Time Problem for the Number of CaMKII Molecules
$$
p_{-}=P(\Theta_b\leq a\leq \Theta_l|postsynaptic spike), \Delta t\leq \Delta t_{+}\\
p_{+}=P(a\geq\Theta_h|postsynaptic spike), \Delta t\leq \Delta t_{+}\\
plus-event:\Delta\geq\Theta_h\\
p_{+}(\varepsilon)=\int_{\Theta_h}^{\infty}\rho(a)da=(1-\varepsilon)\int_{\Theta_h}^{\infty}\rho_0(a)da + \varepsilon\int_{\Theta_h}^{\infty}\rho_0(a-\Delta a)da\\
=(1-\varepsilon)(1-\int_0^{\Theta_h}\rho_0(a)da)+\varepsilon=1 + (\varepsilon-1)\frac{C}{r}\Theta_h^{r}\\
minus-event:\Theta_b<\Theta_l<\Theta_h\leq \Delta a\\
p_{-}(\varepsilon)=\int_{\Theta_b}^{\Theta_l}\rho(a)da=(1-\varepsilon)\int_{\Theta_b}^{\Theta_l}\rho_0(a)da + \varepsilon\int_{\Theta_b}^{\Theta_l}\rho_0(a-\Delta a)da\\
=(1-\varepsilon)\frac{C}{r}(\Theta_l^r-\Theta_b^r)
$$

In [11]:
# parameters setting
Delta_t_plus = 20  # the range of teporal distance
Theta_h = np.exp(-Delta_t_plus/tau_nmda) # the highest threshold of calcium influx - plus event
Theta_l = 0.75*Theta_h # the low threshold of calcium influx - no man land
Theta_b = 0.3*Theta_h # the base threshold of calcium influx - minus event
prob_p = 0.01 # the probability of each CaMKII molecule to be activated
prob_q = 0.01 # the probability of each CaMKII molecule to be deactivated
# p.q without experiment reference. The smaller these values, the narrower the distribution of active CAMKII molecules
X_m = N*0.8 # the minimal amount of active CAMKII molecules for synapse maturation by insertion of new AMPA receptors
X_d = N*0.3 # the minimal amount of the synapse pruning, X_d < X_m.


In [ ]:
def CAMKII_bistable(x, a, p, N, Theta_h, Theta_b, Theta_l):
    '''
    input:
    x is the number of active CAMKII
    a is the Ca influx
    p is the probability of phosphorylate
    N is the number of all CAMKII
    output: 
    updated x
    '''
    if a > Theta_h:
        x  = x + np.random.binomial(N-x, p)
    elif a > Theta_b and a <= Theta_l:
        x = x - np.random.binomial(x, p)
    return x

def probability_plus_event(epsilon):
    '''
    input:
    correlation efficient: epsilon
    output:
    probability of plus-event: p_p 
    '''
    r = v_i*tau_nmda
    C = np.exp(-r*gamma_0)/gamma(gamma_0)
    p_p = 1 + (epsilon - 1)*C/r*Theta_h**r
    return p_p

def probability_minus_event(epsilon):
    '''
    input:
    correlation efficient: epsilon
    output:
    probability of plus-event: p_p 
    '''
    r = v_i*tau_nmda
    C = np.exp(-r*gamma_0)/gamma(gamma_0)
    p_m = (1 - epsilon)*C/r*(Theta_l**r-Theta_b**r)
    return p_m

def equilibrium_active_molecules(p_p, p_m, prob_p, prob_q, N):
    return N*p_p*prob_p/(p_p*prob_p + p_m*prob_q)


## Rate Homeostasis by Synaptic Pruning

- If new synapses are created with a constant rate the input connectivity to the neuron has a stable fi xed point at k* where synaptic death is just compensated by synapse creation. The firing rate of the neuron assumes a corresponding fi xed fi ring rate ν*o. 
$$
C_{io}(\tau)=v_iv_o+c_{io}(\tau)\\
Pr(t_{pre}\in[t-\tau, t-\tau+d\tau]|post\quad spike\quad at\quad t)=v_id\tau+\frac{1}{v_o}c_{io}(\tau)d\tau\\
\frac{dk}{dt}=-k\eta_1(v_o(k),k)\\
k(0)=k_0
$$

## Cooperation and Competition by Spatial Input Correlation

- the calcium based correlation detection mechanism naturally leads to cooperation between synapses, which stabilizes coactive inputs.

$$
\frac{dn_p}{dt}=-n_p\eta_{1,possion}(v_o(n_p,n_c),n_p,n_c)\\
\frac{dn_c}{dt}=-n_c\eta_{1,mip}(v_o(n_p,n_c),n_p,n_c)\\
n_p(0)=n_{p,0},\quad n_c(0)=n_{c,0}
$$

# Appendix

## Probability distribution for the number of active CaMKII molecules
- Markov process
$$
P(x,y) = H(y-x)\sim{p}_{+}B(y-x|p,N-x)+H(x-y)\sim{p}_{-}B(x-y|q,x)\\
\rho(y) = \sum_{x=0}^{N}\rho(x)P(x,y),\quad 0 = \sum_{x=0}^{N}\rho(x)(P(x,t)-\delta(x,y))\\
\delta_{y,0}= \sum_{x=0}^{N}\rho(x)(P(x,t)-\delta(x,y)+\delta_{y,0})\\
\mu(t+\Delta t)=\langle x(t+\Delta t )\rangle\\
\frac{d\mu(t)}{dt}=\lim_{\Delta t\to 0}\frac{\langle x(t+\Delta t)\rangle-\langle x(t)\rangle}{\Delta t}=\lambda\tilde{p}_{+}Np-\mu(t)\lambda(\tilde{p}_{+}p+\tilde{p}_{-}q)\\
\mu_{eq}=\langle x\rangle_{eq}=N\frac{\tilde{p}_{+}p}{\tilde{p}_{+}p+\tilde{p}_{-}q}\\
\mu(t)=\mu(0)e^{-t\lambda(\tilde{p}_{+}p+\tilde{p}_{-}q)}+\mu_{eq}(1-e^{-t\lambda(\tilde{p}_{+}p+\tilde{p}_{-}q)})\\
\tau_{relax}=\frac{1}{\lambda(\tilde{p}_{+}p+\tilde{p}_{-}q)}
$$
$\sigma_{eq}$ can be determined Analogously, thus approximating as a Gaussian

## Mean first passage time problem for the number of CaMKII molecules
$$
\frac{\partial \rho}{\partial t}=-\frac{\partial}{\partial x}S(x)\rho
$$
$S(x)$ denotes the probability flux operator. The process can be desribed as a diffusion and the equilibrium distribution is given by a Gaussian $\rho_0(x)=N(\mu_{eq},\sigma_{eq}^2)$
$$
0 = S(x)\rho_0(x)\to S(x)=D(-(x-\mu_{eq})-\sigma^2_{eq}\frac{\partial}{\partial x})
$$
$D$ control the time scale
$$
A_1(x)=-D(x-\mu_{eq}),\quad D = \lambda(\tilde{p}_{+}p+\tilde{p}_{-}q)
$$
To simplify, rescale the variable $y=\frac{x-\mu_{eq}}{\sigma_{eq}}$. Then end with confluent hypergeometric functions in Quantum mechanics.

## Algorithmic implementation of synapse maturation and synapse death